<a href="https://colab.research.google.com/github/avrilstihler/Agentes-de-IA/blob/main/Imers%C3%A3o_IA_Alura_%2B_Google_Gemini_Agentes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%pip -q install google-genai

In [3]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [4]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [21]:
# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

# Pergunta pro modelo quando é a próxima imersão de IA
resposta = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {resposta.text}"))

Resposta:
 A Alura ainda não divulgou as datas da próxima edição da Imersão IA com Google Gemini. A última edição foi em fevereiro de 2024.

Para ficar por dentro das novidades e não perder a próxima Imersão, recomendo:

*   **Seguir a Alura nas redes sociais:** Instagram, LinkedIn, Twitter e YouTube.
*   **Inscrever-se na newsletter da Alura:** Assim, você recebe todas as novidades diretamente no seu e-mail.
*   **Ficar de olho no site da Alura:** A página da Imersão IA com Google Gemini ([https://www.alura.com.br/imersao/ia-google-gemini](https://www.alura.com.br/imersao/ia-google-gemini)) será atualizada com as informações da próxima edição assim que forem divulgadas.

In [22]:
# Pergunta ao Gemini uma informação utilizando a busca do Google como contexto

response = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
    config={"tools": [{"google_search": {}}]}
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Resposta:
 A próxima Imersão IA com Google Gemini da Alura aconteceu entre os dias 12 e 16 de maio de 2025. As inscrições para esta edição foram encerradas em 11 de maio de 2025.

In [23]:
# Exibe a busca
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Busca realizada: ['Próxima Imersão IA com Google Gemini Alura', 'Alura Imersão IA Google Gemini']
Páginas utilizadas na resposta: starten.tech, tecmundo.com.br, alura.com.br



In [25]:
# Instala o Framework de agentes do Google
!pip install -q google-adk

In [26]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [27]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [28]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [29]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################
def agente_buscador(topico, data_de_hoje):

    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um assistente de pesquisa. A sua tarefa é usar a ferramenta de busca do google (google_search)
        para recuperar as últimas notícias de lançamentos muito relevantes sobre o tópico abaixo.
        Foque em no máximo 5 lançamentos relevantes, com base na quantidade e entusiasmo das notícias sobre ele.
        Se um tema tiver poucas notícias ou reações entusiasmadas, é possível que ele não seja tão relevante assim
        e pode ser substituído por outro que tenha mais.
        Esses lançamentos relevantes devem ser atuais, de no máximo um mês antes da data de hoje.
        """,
        description="Agente que busca informações no Google",
        tools=[google_search]
    )

    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"

    lancamentos = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos

In [31]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um planejador de conteúdo, especialista em redes sociais. Com base na lista de
        lançamentos mais recentes e relevantes buscador, você deve:
        usar a ferramenta de busca do Google (google_search) para criar um plano sobre
        quais são os pontos mais relevantes que poderíamos abordar em um post sobre
        cada um deles. Você também pode usar o (google_search) para encontrar mais
        informações sobre os temas e aprofundar.
        Ao final, você irá escolher o tema mais relevante entre eles com base nas suas pesquisas
        e retornar esse tema, seus pontos mais relevantes, e um plano com os assuntos
        a serem abordados no post que será escrito posteriormente.
        """,
        description="Agente que planeja posts",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [32]:
######################################
# --- Agente 3: Redator do Post --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Redator Criativo especializado em criar posts virais para redes sociais.
            Você escreve posts para a empresa Alura, a maior escola online de tecnologia do Brasil.
            Utilize o tema fornecido no plano de post e os pontos mais relevantes fornecidos e, com base nisso,
            escreva um rascunho de post para Instagram sobre o tema indicado.
            O post deve ser engajador, informativo, com linguagem simples e incluir 2 a 4 hashtags no final.
            """,
        description="Agente redator de posts engajadores para Instagram"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [33]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em posts para redes sociais, com foco no Instagram.
            Por ter um público jovem, entre 18 e 30 anos, use um tom de escrita adequado.
            Revise o rascunho de post de Instagram abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Se o rascunho estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente revisor de post para redes sociais."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [34]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: ")

# Inserir lógica do sistema de agentes ################################################
if not topico:
    print("Você esqueceu de digitar o tópico!")
else:
    print(f"Maravilha! Vamos então criar o post sobre novidades em {topico}")

    lancamentos_buscados = agente_buscador(topico, data_de_hoje)
    print("\n--- 📝 Resultado do Agente 1 (Buscador) ---\n")
    display(to_markdown(lancamentos_buscados))
    print("--------------------------------------------------------------")

    plano_de_post = agente_planejador(topico, lancamentos_buscados)
    print("\n--- 📝 Resultado do Agente 2 (Planejador) ---\n")
    display(to_markdown(plano_de_post))
    print("--------------------------------------------------------------")

    rascunho_de_post = agente_redator(topico, plano_de_post)
    print("\n--- 📝 Resultado do Agente 3 (Redator) ---\n")
    display(to_markdown(rascunho_de_post))
    print("--------------------------------------------------------------")

    post_final = agente_revisor(topico, rascunho_de_post)
    print("\n--- 📝 Resultado do Agente 4 (Revisor) ---\n")
    display(to_markdown(post_final))
    print("--------------------------------------------------------------")

🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀
❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: arte e tecnologia
Maravilha! Vamos então criar o post sobre novidades em arte e tecnologia

--- 📝 Resultado do Agente 1 (Buscador) ---



> Para encontrar os lançamentos mais relevantes na área de arte e tecnologia do último mês, farei algumas buscas no Google.
> 
> 
> Com base nas notícias e tendências de arte e tecnologia de abril e maio de 2025, aqui estão alguns lançamentos e eventos relevantes:
> 
> 1.  **10º Congresso Internacional de Arte, Ciência e Tecnologia e Seminário de Artes Digitais (CIACT-SAD 2025):** Este evento acontecerá de 26 a 28 de maio de 2025 no Centro Cultural da UFMG, em Belo Horizonte. O tema desta edição é "Desconhecido", e o congresso reunirá pesquisadores, artistas e especialistas para debater as interseções entre arte, ciência e tecnologia.
> 
> 2.  **Tendências de Arte Digital em 2025:** Estilos como vaporwave/retrofuturismo, arte 3D minimalista e colagem digital experimental estão em alta. Ferramentas como Blender, Nomad Sculpt e Cinema 4D são populares para a criação de arte digital.
> 
> 3.  **Uso de Inteligência Artificial na Arte:** A IA está sendo cada vez mais utilizada como ferramenta para expandir o campo de criação dos artistas, auxiliando na geração de imagens, pinturas, esculturas, música e poesia.
> 
> 4.  **Realidade Aumentada e Virtual em Exposições:** A RA e a RV estão transformando a forma como o público interage com a arte, permitindo experiências mais imersivas e interativas em museus e galerias.
> 
> 5.  **MAB Educativo:** O Museu de Arte de Brasília (MAB) está promovendo atividades especiais durante a 23ª Semana Nacional de Museus, com foco na interseção entre arte, tecnologia, curadoria e memória. As atividades incluem visitas mediadas escolares, oficinas aos finais de semana e mediação bilíngue.
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 2 (Planejador) ---



> Ok, com base nos lançamentos e eventos que você listou, vou criar um plano para posts sobre cada um deles, identificando os pontos mais relevantes e buscando informações adicionais para aprofundar os temas.
> 
> **1. 10º CIACT-SAD 2025:**
> 
> *   **Pontos Relevantes:**
>     *   Tema "Desconhecido": O que significa esse tema no contexto da arte, ciência e tecnologia? Como ele se manifesta nas obras e discussões do congresso?
>     *   Local e data: O evento está acontecendo no Centro Cultural da UFMG, um espaço importante para a cultura em Belo Horizonte.
>     *   Público: O congresso reúne pesquisadores, artistas e especialistas. Quais são as diferentes perspectivas que esses grupos trazem para a discussão?
> *   **Assuntos para o Post:**
>     1.  Introdução ao CIACT-SAD 2025: O que é o congresso, qual sua importância e histórico.
>     2.  Destaque para o tema "Desconhecido": Exploração do conceito e sua relevância no cenário atual.
>     3.  Entrevistas/Quotes de participantes: Coletar opiniões de artistas, pesquisadores e especialistas sobre o tema e o evento.
>     4.  Cobertura do evento: Fotos e vídeos das apresentações, instalações e discussões.
>     5.  Informações práticas: Como participar, programação completa, etc.
> *   **Google Search:**
>     
> **2. Tendências de Arte Digital em 2025:**
> 
> *   **Pontos Relevantes:**
>     *   Estilos em alta: Vaporwave/retrofuturismo, arte 3D minimalista e colagem digital experimental. O que caracteriza cada um desses estilos? Quais são suas origens e influências?
>     *   Ferramentas populares: Blender, Nomad Sculpt e Cinema 4D. Quais são as vantagens e desvantagens de cada ferramenta? Para que tipo de projeto cada uma é mais adequada?
> *   **Assuntos para o Post:**
>     1.  Visão geral das tendências: Explicar o que são as tendências de arte digital e por que elas são importantes.
>     2.  Exploração dos estilos: Apresentar cada estilo (vaporwave, arte 3D minimalista, colagem digital experimental) com exemplos visuais e descrições detalhadas.
>     3.  Comparativo de ferramentas: Analisar Blender, Nomad Sculpt e Cinema 4D, destacando suas características e funcionalidades.
>     4.  Tutoriais básicos: Criar pequenos tutoriais ou dicas para iniciantes em cada uma das ferramentas.
> *   **Google Search:**
>     
> **3. Uso de Inteligência Artificial na Arte:**
> 
> *   **Pontos Relevantes:**
>     *   A IA como ferramenta criativa: De que maneiras a IA está expandindo o campo da criação artística? Quais são os benefícios e desafios de usar a IA como ferramenta?
>     *   Exemplos de aplicações: Geração de imagens, pinturas, esculturas, música e poesia. Quais são os artistas que estão utilizando a IA de forma inovadora? Quais são as obras mais notáveis criadas com IA?
> *   **Assuntos para o Post:**
>     1.  Introdução à IA na arte: O que é IA generativa e como ela está sendo utilizada por artistas.
>     2.  Aplicações práticas: Apresentar exemplos de obras de arte criadas com IA em diferentes mídias.
>     3.  Entrevistas com artistas: Coletar depoimentos de artistas que utilizam IA em seu processo criativo.
>     4.  Discussão sobre ética e autoria: Abordar as questões éticas relacionadas ao uso da IA na arte, como autoria, originalidade e o papel do artista.
> *   **Google Search:**
>     
> **4. Realidade Aumentada e Virtual em Exposições:**
> 
> *   **Pontos Relevantes:**
>     *   Transformação da experiência do público: Como a RA e a RV estão mudando a forma como as pessoas interagem com a arte? Quais são os benefícios de usar essas tecnologias em museus e galerias?
>     *   Exemplos de exposições: Quais museus e galerias estão utilizando a RA e a RV de forma inovadora? Quais são as obras de arte que se beneficiam dessas tecnologias?
> *   **Assuntos para o Post:**
>     1.  Introdução à RA e RV na arte: O que são RA e RV e como elas estão sendo aplicadas em exposições de arte.
>     2.  Exemplos de uso: Apresentar casos de museus e galerias que utilizam RA e RV para criar experiências imersivas.
>     3.  Benefícios para o público: Explicar como a RA e a RV podem tornar a arte mais acessível, interativa e envolvente.
>     4.  Desafios e oportunidades: Discutir os desafios de implementar a RA e a RV em exposições de arte, como custos, acessibilidade e a necessidade de treinamento para os funcionários do museu.
> *   **Google Search:**
>     


--------------------------------------------------------------

--- 📝 Resultado do Agente 3 (Redator) ---



> **Rascunho de Post para Instagram**
> 
> 🎨 Tecno-artistas, uni-vos! 🚀✨
> 
> Explorando as fronteiras da criatividade, a Alura te leva para um universo onde arte e tecnologia se fundem! Prepare-se para:
> 
> 1️⃣ Desvendar os mistérios do 10º CIACT-SAD 2025 e seu tema "Desconhecido". 🤔 O que será que os artistas e pesquisadores estão tramando?
> 
> 2️⃣ Dominar as tendências de arte digital que vão bombar em 2025: Vaporwave, 3D minimalista e colagens experimentais! 💥 E claro, Blender, Nomad Sculpt e Cinema 4D serão seus melhores amigos.
> 
> 3️⃣ Mergulhar no mundo da IA generativa e descobrir como ela está revolucionando a criação de imagens, músicas e muito mais! 🤖 Será que os robôs vão dominar a arte?
> 
> 4️⃣ Viajar pelas exposições do futuro com Realidade Aumentada e Virtual! 🖼️ + 📱 = 🤯 Prepare-se para interagir com a arte de um jeito totalmente novo.
> 
> E aí, qual dessas tendências te deixou mais hypado? Conta pra gente nos comentários! 👇
> 
> #Alura #ArteTecnologica #Inovacao #Criatividade
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 4 (Revisor) ---



> O rascunho está ótimo, mas podemos deixá-lo ainda mais interessante e engajador para o público do Instagram, com algumas sugestões:
> 
> *   **Tom:** O tom está bom, mas podemos adicionar mais emojis e usar uma linguagem ainda mais próxima da utilizada na internet.
> *   **Chamada para Ação (CTA):** Podemos deixar a chamada para ação mais direta e convidativa.
> 
> **Sugestões de Melhorias:**
> 
> 🎨 E aí, artistas e devs criativos! 🚀✨
> 
> Bora explorar um mundo onde a arte e a tecnologia se encontram?! A Alura te leva nessa! 🤯
> 
> Vem com a gente pra:
> 
> 1️⃣ Desvendar os segredos do 10º CIACT-SAD 2025 e o tema "Desconhecido". 🤔 O que será que vem por aí?
> 
> 2️⃣ Ficar por dentro das tendências de arte digital que vão BOMBAR em 2025: Vaporwave, 3D minimalista e colagens experimentais! 💥 Se liga: Blender, Nomad Sculpt e Cinema 4D serão seus MELHORES AMIGOS! 😉
> 
> 3️⃣ Entrar de cabeça no mundo da IA generativa e ver como ela está mudando a criação de imagens, músicas e tudo mais! 🤖 Será que os robôs vão dominar a arte? 🤷
> 
> 4️⃣ Dar um rolê pelas exposições do futuro com Realidade Aumentada e Virtual! 🖼️ + 📱 = 🤯 Prepare-se para VIVER a arte de um jeito totalmente novo.
> 
> E aí, qual dessas tendências te deixou mais no hype? 🔥 Conta pra gente nos comentários! 👇 Queremos saber TUDO!
> 
> \#Alura \#ArteTecnologica \#Inovacao \#Criatividade \#Tecnologia \#ArteDigital #futurodaarte
> 


--------------------------------------------------------------
